<a href="https://colab.research.google.com/github/yashlal/Deepfake-Microbiomes/blob/main/NetGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import torch.optim as optim
import time
from math import sqrt
import matplotlib.pyplot as plt

# select CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if str(device) == 'cuda:0':
	print('CUDA device selected!')
elif str(device) == 'cpu':
	print('CUDA device not available. CPU selected')

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(462, 462*5)
        self.fc2 = nn.Linear(462*5, 462*5)
        self.fc3 = nn.Linear(462*5, 462*10)
        self.fc4 = nn.Linear(462*10, 231*461)
        self.dropout = nn.Dropout(0.1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
       	x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

net = MyNet().to(device)
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=1e-4)

loss_v = []

for n in range(1,8):
    d_PATH = f'drive/MyDrive/YashData/{n}'
    f = open(d_PATH, 'rb')
    data = pickle.load(f)

    for i in range(1000):
        optimizer.zero_grad()

        input = torch.from_numpy(data[i][1]).float().to(device)
        true_y = torch.FloatTensor(data[i][0]).to(device)

        output = net(input).to(device)

        loss = criterion(output, true_y).to(device)
        loss_v.append(loss.item())
        loss.backward()

        optimizer.step()

        print(f'File {n}: Epoch {i}: {sqrt(loss.item())}')

PATH = 'model.pth'
torch.save(net.state_dict(), PATH)

plt.plot(loss_v)
plt.savefig('Loss.png')
plt.show()


KeyboardInterrupt: ignored